In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import ccxt
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objs as go


In [2]:
copytrader_2_trades = pd.read_csv('trades_3871399360910422273.csv')
copytrader_2_positions = pd.read_csv('positions_3871399360910422273.csv')

In [3]:
copytrader_2_trades.drop('Unnamed: 0', axis=1, inplace=True)
copytrader_2_positions.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
copytrader_2_trades['time'] = pd.to_datetime(copytrader_2_trades['time'], unit = 'ms')
copytrader_2_trades['time'] = pd.to_datetime(copytrader_2_trades['time'], unit = 'ms')
copytrader_2_positions['opened'] = pd.to_datetime(copytrader_2_positions['opened'], unit = 'ms')
copytrader_2_positions['closed'] = pd.to_datetime(copytrader_2_positions['closed'], unit = 'ms')
copytrader_2_positions['updateTime'] = pd.to_datetime(copytrader_2_positions['updateTime'], unit = 'ms')

In [32]:
copytrader_2_positions

,id,symbol,type,opened,closed,avgCost,avgClosePrice,closingPnl,maxOpenInterest,closedVolume,isolated,side,status,updateTime
0,5778622,1000SHIBUSDT,UM,2024-01-08 17:57:11.230,2024-01-08 18:01:15.036,0.009276,0.009345,0.120612,1748.0,1748.0,Cross,Long,All Closed,2024-01-08 18:01:15.185
1,5780298,1000SHIBUSDT,UM,2024-01-08 18:02:18.814,2024-01-08 18:23:25.200,0.009318,0.009388,0.121870,1741.0,1741.0,Cross,Long,All Closed,2024-01-08 18:23:25.375
2,5784298,1000SHIBUSDT,UM,2024-01-08 18:24:18.278,2024-01-08 18:38:46.736,0.009408,0.009478,0.120680,1724.0,1724.0,Cross,Long,All Closed,2024-01-08 18:38:46.887
3,5788014,1000SHIBUSDT,UM,2024-01-08 18:39:11.663,2024-01-08 19:34:04.275,0.009468,0.009539,0.121623,1713.0,1713.0,Cross,Long,All Closed,2024-01-08 19:34:04.480
4,5797669,1000SHIBUSDT,UM,2024-01-08 19:35:18.313,2024-01-09 04:45:36.851,0.009425,0.009496,0.405878,5723.0,5723.0,Cross,Long,All Closed,2024-01-09 14:20:41.198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2445,32227996,DOGEUSDT,UM,2024-04-25 07:35:19.932,2024-04-25 11:10:42.023,0.148486,0.149150,0.547520,824.0,824.0,Cross,Long,All Closed,2024-04-25 11:10:42.192
2446,32254273,DOGEUSDT,UM,2024-04-25 11:13:38.318,2024-04-25 11:18:07.112,0.148780,0.149450,0.156110,233.0,233.0,Cross,Long,All Closed,2024-04-25 11:18:07.607
2447,32267701,DOGEUSDT,UM,2024-04-25 12:41:23.128,2024-04-25 14:07:34.504,0.149470,0.150140,0.155440,232.0,232.0,Cross,Long,All Closed,2024-04-25 14:07:34.810
2448,32291678,DOGEUSDT,UM,2024-04-25 14:53:32.671,2024-04-25 15:02:11.843,0.149330,0.150000,0.156110,233.0,233.0,Cross,Long,All Closed,2024-04-25 15:02:12.051


# Func

In [29]:
def data_dow(symbol, timeframe, start_date, end_date):
    exchange = ccxt.binance()
    since = exchange.parse8601(start_date.strftime('%Y-%m-%dT%H:%M:%S'))
    until = exchange.parse8601(end_date.strftime('%Y-%m-%dT%H:%M:%S'))
    all_ohlcv = []

    while since < until:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since)
        if not ohlcv:
            break
        all_ohlcv.extend(ohlcv)
        since = ohlcv[-1][0] + 1

    df = pd.DataFrame(all_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    return df


In [27]:
def long_short(data, cur_pair):
    data = data[data['symbol'] == cur_pair]
    data_long = data[data['side'] == 'Long']
    data_short = data[data['side'] == 'Short']

    return [data_long, data_short]

In [34]:
def price_graph(data, bot_long, bot_short):
    candlestick = go.Candlestick(
                            x=data['timestamp'],
                            open=data['open'],
                            high=data['high'],
                            low=data['low'],
                            close=data['close'])
    
    bot_positions_long = go.Scatter(x=bot_long['opened'], y=bot_long['avgCost'], mode='markers', name='Bot Positions', marker_color = 'blue')
    bot_positions_short = go.Scatter(x=bot_short['opened'], y=bot_short['avgCost'], mode='markers', name='Bot Positions', marker_color = 'yellow')
    
    bot_positions_long_c = go.Scatter(x=bot_long['closed'], y=bot_long['avgClosePrice'], mode='markers', name='Bot Positions', marker_color = 'purple')
    bot_positions_short_c = go.Scatter(x=bot_short['closed'], y=bot_short['avgClosePrice'], mode='markers', name='Bot Positions', marker_color = 'black')

    fig = go.Figure(data=[candlestick, bot_positions_long, bot_positions_short, bot_positions_long_c, bot_positions_short_c])
    
    fig.update_layout(
        width=2400,
        height=800,  
    )
    fig.show()

# Кольори точок: 
#   blue - відкрита позиція на лонг
#   purple - закрита позиція на лонг
#   yellow - відкрита позиція на шорт
#   black - закрита позиція на шорт

# Очистка даних

## Дані про трейди

In [21]:
copytrader_2_trades.describe()

,time,price,fee,quantity,realizedProfit,qty
count,5600,5600.000000,5600.000000,5600.000000,5600.000000,5600.000000
mean,2024-03-18 13:10:58.451071488,0.446963,-0.017475,84.883999,0.091203,525.471041
min,2024-02-27 22:37:36,0.012146,-0.983573,0.552762,-269.673258,0.290000
25%,2024-03-05 18:49:40.249999872,0.131105,-0.012304,27.953082,0.000000,198.000000
50%,2024-03-13 08:46:51,0.159290,-0.007818,39.075560,0.000000,243.000000
75%,2024-03-30 16:47:08,0.186783,-0.005602,61.520450,0.177705,431.250000
max,2024-04-25 15:51:05,8.995000,-0.000111,2813.156500,56.194988,18646.000000
std,NaN,1.140170,0.040632,173.357993,5.010048,1015.482676


In [22]:
copytrader_2_trades.isna().sum()

time                   0
symbol                 0
side                   0
price                  0
fee                    0
feeAsset               0
quantity               0
quantityAsset          0
realizedProfit         0
realizedProfitAsset    0
baseAsset              0
qty                    0
positionSide           0
activeBuy              0
dtype: int64

## Дані про позиції

In [23]:
copytrader_2_positions.describe()

,id,opened,closed,avgCost,avgClosePrice,closingPnl,maxOpenInterest,closedVolume,updateTime
count,2.450000e+03,2450,2450,2450.000000,2450.000000,2450.000000,2450.000000,2450.000000,2450
mean,1.840363e+07,2024-03-07 03:19:31.584987136,2024-03-07 05:46:48.095714560,1.142285,1.156123,0.289022,793.261200,823.808029,2024-03-07 07:28:16.725013248
min,5.778622e+06,2024-01-08 17:57:11.230000,2024-01-08 18:01:15.036000,0.009102,0.009264,-752.659700,0.200000,0.200000,2024-01-08 18:01:15.185000
25%,1.451444e+07,2024-03-01 04:02:25.801250048,2024-03-01 05:33:06.770749952,0.134153,0.134780,0.129280,154.000000,154.000000,2024-03-02 03:05:20.181999872
50%,1.699774e+07,2024-03-06 16:06:00.062499840,2024-03-06 16:15:38.832999936,0.162660,0.163390,0.174735,228.000000,228.000000,2024-03-06 16:15:39.044999936
75%,2.403648e+07,2024-03-24 15:57:52.265749760,2024-03-24 17:21:57.067249920,0.204883,0.205987,0.280743,331.750000,331.850000,2024-03-24 17:21:57.202000128
max,3.229304e+07,2024-04-25 15:05:24.548000,2024-04-25 15:50:50.973000,41.330000,42.214000,135.578720,74757.000000,74757.000000,2024-04-25 15:50:51.139000
std,7.461712e+06,NaN,NaN,4.358720,4.433952,15.644632,2778.495536,2945.300446,NaN


In [24]:
copytrader_2_positions.isna().sum()

id                 0
symbol             0
type               0
opened             0
closed             0
avgCost            0
avgClosePrice      0
closingPnl         0
maxOpenInterest    0
closedVolume       0
isolated           0
side               0
status             0
updateTime         0
dtype: int64

In [5]:
copytrader_2_positions = copytrader_2_positions.sort_values('opened').reset_index(drop=True)

## MaticUsdt

In [19]:
data_trades = copytrader_2_trades[copytrader_2_trades['symbol'] == 'MATICUSDT']

In [25]:
data_position = copytrader_2_positions[copytrader_2_positions['symbol'] == 'MATICUSDT']

In [33]:
data_position

,id,symbol,type,opened,closed,avgCost,avgClosePrice,closingPnl,maxOpenInterest,closedVolume,isolated,side,status,updateTime
137,6405170,MATICUSDT,UM,2024-01-12 07:51:47.562,2024-01-12 09:47:52.556,0.905700,0.913200,0.0450,6.0,6.0,Cross,Long,All Closed,2024-01-12 09:47:52.773
154,6478051,MATICUSDT,UM,2024-01-12 15:38:32.963,2024-01-12 20:22:35.147,0.890225,0.897600,0.1770,24.0,24.0,Cross,Long,All Closed,2024-01-12 20:22:35.338
164,6540665,MATICUSDT,UM,2024-01-12 20:35:24.627,2024-01-13 00:22:52.177,0.854675,0.863611,8.9808,1005.0,1005.0,Cross,Long,All Closed,2024-01-13 00:22:52.334
173,6587857,MATICUSDT,UM,2024-01-13 00:27:01.221,2024-01-13 02:02:08.323,0.845850,0.855544,0.9306,96.0,96.0,Cross,Long,All Closed,2024-01-13 02:02:08.638
177,6603980,MATICUSDT,UM,2024-01-13 02:02:54.311,2024-01-13 02:19:19.785,0.854800,0.861900,0.0426,6.0,6.0,Cross,Long,All Closed,2024-01-13 02:19:19.994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2416,31640392,MATICUSDT,UM,2024-04-22 03:49:48.376,2024-04-22 05:56:17.879,0.731800,0.739200,0.4810,65.0,65.0,Cross,Long,All Closed,2024-04-22 05:56:18.022
2417,31659833,MATICUSDT,UM,2024-04-22 05:56:41.820,2024-04-22 12:25:33.658,0.739300,0.746700,0.4736,64.0,64.0,Cross,Long,All Closed,2024-04-22 12:25:33.836
2421,31704224,MATICUSDT,UM,2024-04-22 12:25:40.332,2024-04-24 04:46:59.043,0.746300,0.753800,0.4725,63.0,63.0,Cross,Long,All Closed,2024-04-24 04:46:59.208
2436,31989950,MATICUSDT,UM,2024-04-24 04:47:08.458,2024-04-24 04:58:20.655,0.754800,0.762400,0.4788,63.0,63.0,Cross,Long,All Closed,2024-04-24 04:58:20.751


In [35]:
matic_bot_long, matic_bot_short = long_short(copytrader_2_positions, 'MATICUSDT')[0], long_short(copytrader_2_positions, 'MATICUSDT')[1]

symbol = 'MATIC/USDT'
timeframe = '1h'  # Годинні свічки
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 5, 1)

df = data_dow(symbol, timeframe, start_date, end_date)

price_graph(df, matic_bot_long, matic_bot_short)

## Аналіз бота

1.  Ну поки що можна сказати, бот точно більшість позицій відкриває зі стоп-лосом, і він явно ставить його на цю ж ціну, при якій він купив\продав дану позицію  
2.  Також, є ідея, що після впевненого росту/падіння(приблизно 2-3 свічки підряд в 1 сторону), бот виставляє декілька позицій і закриває їх зразу, якщо позиція стає неприбутковою
3.  Помітно що, різниця між відкриттям нових позицій має бути більша за 0.375% в якусь сторону для відкриття нової позиції(при чому, коливання має бути стабільне(тобто має бути 2-3 свічки в одну сторону + різниця позиції 0.375%))

4.  Якщо бот бачить, що після 3-ох свічок, позиція закрилась на + Pnl, і ціна росте\падає, він знову відкриває позиції - при чому на 1 свічку іде 1 позиція.

In [43]:
data_position['diff'] = abs(data_position['avgClosePrice'] - data_position['avgCost'])

C:\Users\MSI\AppData\Local\Temp\ipykernel_2136\2760677635.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
data_position

,id,symbol,type,opened,closed,avgCost,avgClosePrice,closingPnl,maxOpenInterest,closedVolume,isolated,side,status,updateTime,diff
137,6405170,MATICUSDT,UM,2024-01-12 07:51:47.562,2024-01-12 09:47:52.556,0.905700,0.913200,0.0450,6.0,6.0,Cross,Long,All Closed,2024-01-12 09:47:52.773,0.007500
154,6478051,MATICUSDT,UM,2024-01-12 15:38:32.963,2024-01-12 20:22:35.147,0.890225,0.897600,0.1770,24.0,24.0,Cross,Long,All Closed,2024-01-12 20:22:35.338,0.007375
164,6540665,MATICUSDT,UM,2024-01-12 20:35:24.627,2024-01-13 00:22:52.177,0.854675,0.863611,8.9808,1005.0,1005.0,Cross,Long,All Closed,2024-01-13 00:22:52.334,0.008936
173,6587857,MATICUSDT,UM,2024-01-13 00:27:01.221,2024-01-13 02:02:08.323,0.845850,0.855544,0.9306,96.0,96.0,Cross,Long,All Closed,2024-01-13 02:02:08.638,0.009694
177,6603980,MATICUSDT,UM,2024-01-13 02:02:54.311,2024-01-13 02:19:19.785,0.854800,0.861900,0.0426,6.0,6.0,Cross,Long,All Closed,2024-01-13 02:19:19.994,0.007100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2416,31640392,MATICUSDT,UM,2024-04-22 03:49:48.376,2024-04-22 05:56:17.879,0.731800,0.739200,0.4810,65.0,65.0,Cross,Long,All Closed,2024-04-22 05:56:18.022,0.007400
2417,31659833,MATICUSDT,UM,2024-04-22 05:56:41.820,2024-04-22 12:25:33.658,0.739300,0.746700,0.4736,64.0,64.0,Cross,Long,All Closed,2024-04-22 12:25:33.836,0.007400
2421,31704224,MATICUSDT,UM,2024-04-22 12:25:40.332,2024-04-24 04:46:59.043,0.746300,0.753800,0.4725,63.0,63.0,Cross,Long,All Closed,2024-04-24 04:46:59.208,0.007500
2436,31989950,MATICUSDT,UM,2024-04-24 04:47:08.458,2024-04-24 04:58:20.655,0.754800,0.762400,0.4788,63.0,63.0,Cross,Long,All Closed,2024-04-24 04:58:20.751,0.007600


In [42]:
data_position['diff'].describe()

count    266.000000
mean       0.010078
std        0.005414
min        0.003750
25%        0.009000
50%        0.010200
75%        0.010952
max        0.093510
Name: diff, dtype: float64